In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(1, '/home/georgeos/Documents/SyMBac/') # Not needed if you installed SyMBac using pip

from SyMBac.simulation import Simulation
from SyMBac.PSF import PSF_generator
from SyMBac.renderer import Renderer
from SyMBac.PSF import Camera
from SyMBac.misc import get_sample_images
real_image = get_sample_images()["E. coli 100x"]

SyntaxError: invalid syntax (cell_simulation.py, line 17)

In [ ]:
from SyMBac.cell_geometry import get_vertices
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cell_verts = get_vertices(6,1,np.pi/2,40)
x_verts, y_verts = [_[0] for _ in cell_verts], [_[1] for _ in cell_verts]

In [ ]:
plt.scatter(x_verts, y_verts)

In [ ]:
from SyMBac.cell_simulation import run_simulation, create_space, trench_creator, step_and_update
from SyMBac.cell import Cell
import numpy as np
from pymunk.pyglet_util import DrawOptions
import pymunk
import pyglet

In [ ]:
gravity = 0 
pix_mic_conv = 0.065
trench_length = 20
trench_width = 1.1
cell_max_length = 3
cell_width = 1
max_length_var = 0
width_var = 0
lysis_p = 0
show_window = True
phys_iters = 0
sim_length = 1000
save_dir = "/tmp/"

In [ ]:

space = create_space()
space.gravity = 0, gravity  # arbitrary units, negative is toward trench pole
#space.iterations = 1000
#space.damping = 0
#space.collision_bias = 0.0017970074436457143*10
space.collision_slop = 0.
dt = 1 / 100  # time-step per frame
pix_mic_conv = 1 / pix_mic_conv  # micron per pixel
scale_factor = pix_mic_conv * 3  # resolution scaling factor

trench_length = trench_length * scale_factor
trench_width = trench_width * scale_factor
trench_creator(trench_width, trench_length, (35, 0), space)  # Coordinates of bottom left corner of the trench

cell1 = Cell(
    length=cell_max_length/2 * scale_factor,
    width=cell_width * scale_factor,
    resolution=60,
    position=(50, 50), #x.y
    angle=np.pi/4,
    space=space,
    dt= dt,
    growth_rate_constant=1,
    max_length=cell_max_length * scale_factor,
    max_length_mean=cell_max_length * scale_factor,
    max_length_var=max_length_var * np.sqrt(scale_factor),
    width_var=width_var * np.sqrt(scale_factor),
    width_mean=cell_width * scale_factor,
    parent=None,
    lysis_p=lysis_p
)

if show_window:

    window = pyglet.window.Window(700, 700, "SyMBac", resizable=True)
    options = DrawOptions()
    options.shape_outline_color = (10,20,30,40)
    @window.event
    def on_draw():
        window.clear()
        space.debug_draw(options)

    # key press event
    @window.event
    def on_key_press(symbol, modifier):

        # key "E" get press
        if symbol == pyglet.window.key.E:
            # close the window
            window.close()



x = [0]
cell_timeseries = []
cells = [cell1]
if show_window:
    pyglet.clock.schedule_interval(step_and_update, interval=dt, cells=cells, space=space, phys_iters=phys_iters,
                                   ylim=trench_length, cell_timeseries=cell_timeseries, x=x, sim_length=sim_length,
                                   save_dir=save_dir)
    pyglet.app.run()
else:
    for _ in tqdm(range(sim_length+2)):
        step_and_update(
            dt=dt, cells=cells, space=space, phys_iters=phys_iters, ylim=trench_length,
            cell_timeseries=cell_timeseries, x=x, sim_length=sim_length, save_dir=save_dir
        )


# window.close()
# phys_iters = phys_iters
# for x in tqdm(range(sim_length+250),desc="Simulation Progress"):
#    cells = step_and_update(dt=dt, cells=cells, space=space, phys_iters=phys_iters,ylim=trench_length*1.1, cell_timeseries = cell_timeseries, x=x, sim_length = sim_length, save_dir = save_dir)
#    if x > 250:
#        cell_timeseries.append(deepcopy(cells))

In [ ]:
cell_timeseries